In [27]:
import pandas as pd 
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import pmdarima as pm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import grangercausalitytests
from pathlib import Path
from datetime import datetime
from fuzzywuzzy import process
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [28]:
current_file_path = Path.cwd()
root = current_file_path.parent

external_path = root / 'data' / 'external_data' 
processed_path = root / 'data'/ 'processed_data'

In [29]:
rtrs_data = pd.read_csv(processed_path / 'brazil_rtrs.csv', encoding = 'iso-8859-14', decimal=',')
farm_branches = pd.read_csv(processed_path / 'farm_branches.csv', usecols = ['State', 'City', 'Organization'])
b3_data = pd.read_csv(external_path / 'acoes-listadas-b3.csv')
trase = pd.read_excel(external_path / 'trase_soy_2-6-0.xlsx')

In [30]:
def find_similarities(list1, list2, limit=75):
    """
    Finds similar strings between two lists using FuzzyWuzzy.
    
    Parameters:
    - list1: List of strings to compare with the second list.
    - list2: List of strings against which the first list will be compared.
    - threshold: Similarity threshold for considering a string as a match (default 75).
    
    Returns:
    - A list of tuples containing the string from list1, the most similar string from list2 and the similarity score.
    """
    
    similar_results = []

    for item in list1:
        # Find the best match for 'item' in 'list2'
        best_match, score = process.extractOne(item, list2)
        
        # Check if the similarity is above the defined threshold
        if score >= limit:
            similar_results.append(best_match)
            
    return similar_results

In [31]:
#Similarity between Trase and RTRS

amazon_states = [
    "ACRE",
    "AMAPA",
    "AMAZONAS",
    "MARANHAO",
    "MATO GROSSO",
    "PARA",
    "RONDONIA",
    "RORAIMA",
    "TOCANTINS"
]

trase = trase[trase['STATE'].isin(amazon_states)]
rtrs_groups = find_similarities(rtrs_data['Organization'].unique().tolist(), trase['EXPORTER GROUP'].unique().tolist(), limit=93)
rtrs_groups.append('ADM')
rtrs_groups.append('SLC')
trase['IS_RTRS'] = trase['EXPORTER GROUP'].apply(lambda x: 1 if x in rtrs_groups else 0)

In [32]:
trase[['EXPORTER GROUP', 'IS_RTRS']]

,EXPORTER GROUP,IS_RTRS
0,DOMESTIC CONSUMPTION,0
1,DOMESTIC CONSUMPTION,0
2,DOMESTIC CONSUMPTION,0
3,CARGILL,0
4,CARGILL,0
...,...,...
283780,GIRASSOL AGRICOLA,1
283781,GIRASSOL AGRICOLA,1
283782,GIRASSOL AGRICOLA,1
283783,GIRASSOL AGRICOLA,1


In [33]:
trase.rename(columns={'EXPORTER GROUP': 'Group', 
                      'Soy deforestation exposure': 'SoyDefExp',
                      'FOB_USD': 'ExpFOB',
                      'CO2_GROSS_EMISSIONS_SOY_DEFORESTATION_5_YEAR_TOTAL_EXPOSURE': 'CO2Gross',
                      'SOY_EQUIVALENT_TONNES': 'SoyTonnes',
                      'LAND_USE_HA': 'LandUse',
                      'CO2_NET_EMISSIONS_SOY_DEFORESTATION_5_YEAR_TOTAL_EXPOSURE': 'CO2Net',
                      'Soy deforestation risk': 'SoyRisk',
                      'IS_RTRS': 'IsRTRS' ,
                      'STATE': 'State',
                      'MUNICIPALITY OF PRODUCTION': 'City',
                      'YEAR': 'Year'
                      }, inplace=True)

In [34]:
trase = trase.groupby(['Year', 'Group', 'IsRTRS']).agg({'SoyDefExp': 'sum', 
                                        'ExpFOB': 'sum',
                                        'CO2Gross': 'sum',
                                        'CO2Net': 'sum',
                                        'SoyTonnes': 'sum', 
                                        'LandUse':'sum',
                                        'SoyRisk': 'sum',
                                        #'Yield': 'mean',
                                        }).reset_index()

In [35]:
NotRTRS = trase[trase['IsRTRS'] == 0]
RTRS = trase[trase['IsRTRS'] == 1]

In [36]:
ts_notrtrs = NotRTRS.groupby('Year').agg({'SoyTonnes': 'sum', 'LandUse': 'sum'}).reset_index()
ts_notrtrs['Income'] = ts_notrtrs['SoyTonnes'] / ts_notrtrs['LandUse']
ts_rtrs = RTRS.groupby('Year').agg({'SoyTonnes': 'sum', 'LandUse': 'sum'}).reset_index()
ts_rtrs['Income'] = ts_rtrs['SoyTonnes'] / ts_rtrs['LandUse']

In [37]:
def calcular_taxas_crescimento_anual(dados, coluna=None):
    """
    Calcula as taxas de crescimento anual para uma coluna especificada em um DataFrame
    ou para uma lista de valores.
    
    :param dados: DataFrame do pandas ou lista de valores.
    :param coluna: (Opcional) Nome da coluna para a qual calcular as taxas de crescimento, se dados for um DataFrame.
    :return: Uma lista contendo as taxas de crescimento anual.
    """
    taxas_crescimento = []
    
    # Se 'dados' for um DataFrame, utiliza a coluna especificada.
    if isinstance(dados, pd.DataFrame) and coluna is not None:
        valores = dados[coluna].values
    elif isinstance(dados, list):
        valores = dados
    else:
        raise ValueError("Os dados devem ser um DataFrame do pandas com uma coluna especificada ou uma lista de valores.")
    
    for i in range(1, len(valores)):
        valor_atual = valores[i]
        valor_anterior = valores[i - 1]
        taxa_crescimento = ((valor_atual - valor_anterior) / valor_anterior) * 100
        taxas_crescimento.append(taxa_crescimento)
    
    return taxas_crescimento

In [38]:
ts_notrtrs['Tx_Producao'] = [None] + calcular_taxas_crescimento_anual(ts_notrtrs, 'SoyTonnes')
ts_notrtrs['Tx_Produtividade'] = [None] + calcular_taxas_crescimento_anual(ts_notrtrs, 'Income')
ts_rtrs['Tx_Producao'] = [None] + calcular_taxas_crescimento_anual(ts_rtrs, 'SoyTonnes')
ts_rtrs['Tx_Produtividade'] = [None] + calcular_taxas_crescimento_anual(ts_rtrs, 'Income')

ts_producao_total_cadeia = ts_notrtrs['SoyTonnes'] + ts_rtrs['SoyTonnes']
ts_producao_total_cadeia= ts_producao_total_cadeia.to_list()
ts_producao_tx_total_cadeia = calcular_taxas_crescimento_anual(ts_producao_total_cadeia)

In [39]:
ts_notrtrs['MS'] = (ts_notrtrs['SoyTonnes'] / ts_producao_total_cadeia) * 100
#ts_notrtrs = ts_notrtrs.drop(ts_notrtrs.index[-1])
ts_rtrs['MS'] = (ts_rtrs['SoyTonnes'] / ts_producao_total_cadeia) * 100
#ts_rtrs = ts_rtrs.drop(ts_rtrs.index[-1])

In [40]:
ts_rtrs

,Year,SoyTonnes,LandUse,Income,Tx_Producao,Tx_Produtividade,MS
0,2004,5.127570e+07,1.616484e+06,31.720518,NaN,NaN,14.774816
1,2005,9.649110e+07,3.118201e+06,30.944475,88.180941,-2.446503,7.996101
2,2006,2.375114e+08,2.996724e+06,79.257026,146.148550,156.126583,41.884207
3,2007,8.617800e+08,6.286498e+07,13.708427,262.837253,-82.703834,67.569016
4,2008,2.061439e+09,2.522815e+07,81.711861,139.207158,496.070313,74.487066
5,2009,1.720315e+09,6.108541e+07,28.162453,-16.547869,-65.534437,57.590957
6,2010,1.743370e+09,2.030851e+07,85.844329,1.340186,204.818371,54.652990
7,2011,2.778451e+09,3.173409e+06,875.541424,59.372387,919.917605,70.731960
8,2012,2.005579e+09,2.125798e+07,94.344736,-27.816664,-89.224412,51.878176
9,2013,2.162245e+09,2.314746e+07,93.411745,7.811510,-0.988917,66.614229


In [41]:
p0_producao_notrtrs = ts_notrtrs['Tx_Producao'].iloc[:7].mean()
p1_producao_notrtrs = ts_notrtrs['Tx_Producao'].iloc[7:].mean()

p0_producao_rtrs = ts_rtrs['Tx_Producao'].iloc[:7].mean()
p1_producao_rtrs = ts_rtrs['Tx_Producao'].iloc[7:].mean()

p0_producao_amazonia = np.mean(ts_producao_tx_total_cadeia[7:])
p1_producao_amazonia = np.mean(ts_producao_tx_total_cadeia[:7])

p0_produtividade_rtrs = ts_rtrs['Income'].iloc[7:].mean()
p1_produtividade_rtrs = ts_rtrs['Income'].iloc[:7].mean()

p0_produtividade_notrtrs = ts_notrtrs['Income'].iloc[7:].mean()
p1_produtividade_notrtrs = ts_notrtrs['Income'].iloc[:7].mean()

print(f'Produção em periodo 0 da NOTRTRS: {p0_producao_notrtrs:.2f}')
print(f'Produção em periodo 1 da NOTRTRS: {p1_producao_notrtrs:.2f}')

print(f'Produção em periodo 0 da RTRS: {p0_producao_rtrs:.2f}')
print(f'Produção em periodo 1 da RTRS: {p1_producao_rtrs:.2f}')

print(f'Produção em periodo 0 da amazonia: {p0_producao_amazonia:.2f}')
print(f'Produção em periodo 1 da amazonia: {p1_producao_amazonia:.2f}')

print(f'Produtividade em periodo 0 da NOTRTRS: {p0_produtividade_notrtrs:.2f}')
print(f'Produtividade em periodo 1 da NOTRTRS: {p1_produtividade_notrtrs:.2f}')

print(f'Produtividade em periodo 0 da RTRS: {p0_produtividade_rtrs:.2f}')
print(f'Produtividade em periodo 1 da RTRS: {p1_produtividade_rtrs:.2f}')

Produção em periodo 0 da NOTRTRS: 65.81
Produção em periodo 1 da NOTRTRS: 41.31
Produção em periodo 0 da RTRS: 103.53
Produção em periodo 1 da RTRS: -17.60
Produção em periodo 0 da amazonia: -6.39
Produção em periodo 1 da amazonia: 67.78
Produtividade em periodo 0 da NOTRTRS: 96.68
Produtividade em periodo 1 da NOTRTRS: 226.17
Produtividade em periodo 0 da RTRS: 246.96
Produtividade em periodo 1 da RTRS: 50.19


In [42]:
r = p1_producao_amazonia / p0_producao_amazonia

eem_rtrs = r * p0_producao_rtrs

eem_notrtrs = r * p0_producao_notrtrs

In [43]:
ecr_rtrs = p1_producao_rtrs - p0_producao_rtrs - eem_rtrs
ecr_notrtrs = p1_producao_notrtrs - p0_producao_notrtrs - eem_notrtrs

In [44]:
p_rtrs = p1_produtividade_rtrs / p0_produtividade_rtrs
p_notrtrs = p1_produtividade_notrtrs / p0_produtividade_notrtrs

epr_rtrs = p_rtrs * p0_producao_rtrs
epr_notrtrs = p_rtrs * p0_producao_notrtrs

In [45]:
exr_rtrs =  p1_producao_rtrs - p0_producao_rtrs - epr_rtrs - eem_rtrs
exr_notrtrs = p1_producao_notrtrs - p0_producao_notrtrs - epr_notrtrs - eem_notrtrs

In [46]:
print(f'Expansão de mercado da não RTRS: {eem_notrtrs:.2f}, Expansão de Mercado RTRS: {eem_rtrs:.2f}')
print(f'Efeito concorrência da não RTRS: {ecr_notrtrs:.2f}, Efeito Concorrência da RTRS: {ecr_rtrs:.2f}')
print(f'Efeito produtividade da não RTRS: {epr_notrtrs:.2f}, Efeito produtividade da RTRS: {epr_rtrs:.2f}')
print(f'Efeito externalidades diversas da não RTRS: {exr_notrtrs:.2f}, Efeito externalidades diversas da RTRS: {exr_rtrs:.2f}')

Expansão de mercado da não RTRS: -697.96, Expansão de Mercado RTRS: -1097.96
Efeito concorrência da não RTRS: 673.46, Efeito Concorrência da RTRS: 976.84
Efeito produtividade da não RTRS: 13.38, Efeito produtividade da RTRS: 21.04
Efeito externalidades diversas da não RTRS: 660.08, Efeito externalidades diversas da RTRS: 955.79


In [47]:
V_rtrs = exr_rtrs + epr_rtrs + eem_rtrs 
print(V_rtrs, p1_producao_rtrs - p0_producao_rtrs)

-121.12283644049796 -121.122836440498


In [48]:
V_notrtrs = exr_notrtrs + epr_notrtrs + eem_notrtrs 
print(V_notrtrs, p1_producao_notrtrs - p0_producao_notrtrs)

-24.50163490334262 -24.501634903342634


# Mercado de Terras

In [49]:
t_p_rtrs = (p1_producao_rtrs/(8 * p1_produtividade_rtrs)) - (p0_producao_rtrs/(8 * p0_produtividade_rtrs)) 
t_exr_rtrs = exr_rtrs / (8 * p1_produtividade_rtrs)
t_em_rtrs = eem_rtrs / (8 * p1_produtividade_rtrs)
t_pr_rtrs = epr_rtrs / (8 * p1_produtividade_rtrs)

t_p_01_rtrs = t_p_rtrs + t_exr_rtrs + t_em_rtrs + t_pr_rtrs

print(f'Variação no total anual médio de terras requeridas entre os períodos 0 e 1 da RTRS: {t_p_01_rtrs}')

Variação no total anual médio de terras requeridas entre os períodos 0 e 1 da RTRS: -0.3978635801834088


In [53]:
print(t_p_01_rtrs, t_exr_rtrs, t_em_rtrs, t_pr_rtrs, t_p_rtrs)

-0.3978635801834088 2.3803107728891755 -2.7343554212432033 0.05240025628214632 -0.09621918811152735


In [57]:
t_p_notrtrs = (p1_producao_notrtrs/(8 * p1_produtividade_notrtrs)) - (p0_producao_notrtrs/(8 * p0_produtividade_notrtrs)) 
t_exr_notrtrs = exr_notrtrs / (8 * p1_produtividade_notrtrs)
t_em_notrtrs = eem_notrtrs / (8 * p1_produtividade_notrtrs)
t_pr_notrtrs = epr_notrtrs / (8 * p1_produtividade_notrtrs)

t_p_01_notrtrs = t_p_notrtrs + t_exr_notrtrs + t_em_notrtrs + t_pr_notrtrs

print(f'Variação no total anual médio de terras requeridas entre os períodos 0 e 1 da NO RTRS: {t_p_01_notrtrs}')

Variação no total anual médio de terras requeridas entre os períodos 0 e 1 da NO RTRS: -0.07580169297986139


In [58]:
t_p_01_notrtrs > t_p_01_rtrs

True

In [59]:
print(t_p_01_notrtrs, t_exr_notrtrs, t_em_notrtrs, t_pr_notrtrs, t_p_notrtrs)

-0.07580169297986139 0.3648123746499721 -0.3857461993552762 0.007392308823148614 -0.06226017709770594


## Índice de Jevons-Bourlag

In [ ]:
ijb_rtrs = - (t_p_rtrs + t_pr_rtrs + t_exr_rtrs)/t_p_rtrs

print(f'Indice Jevons-Bourlag da RTRS: {ijb_rtrs}')

Indice Jevons-Bourlag da RTRS: 24.28301347078063


In [ ]:
ijb_notrtrs = - (t_p_notrtrs + t_pr_notrtrs + t_exr_notrtrs)/t_p_notrtrs

print(f'Indice Jevons-Bourlag da NO RTRS: {ijb_notrtrs}')

Indice Jevons-Bourlag da NO RTRS: 4.978214338982905
